# Import the Necessary Libraries

In [1]:
# Data and Plotting
import pandas as pd
import numpy as np
import plotly.express as px
from ucimlrepo import fetch_ucirepo 
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, DetCurveDisplay

# Data encoding and Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, Normalizer
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

# Dimensionality Reduction and Clustering Algorithms
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans

from sklearn.utils.class_weight import compute_sample_weight

# Data spliting and Cross Validation and Performance Metrics
from sklearn.model_selection import train_test_split as tts, StratifiedKFold, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_validate, KFold
from sklearn.metrics import recall_score, f1_score, balanced_accuracy_score, roc_auc_score
from sklearn.metrics import  make_scorer,  precision_score, accuracy_score

# models
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.neural_network import MLPClassifier as FNN

from collections import defaultdict as dd
from sklearn.utils import resample

%matplotlib notebook

# Load the Dataset

In [2]:
data = pd.read_csv("Thyroid_Diff.csv")

X = data.drop("Recurred", axis=1)
y = data["Recurred"]
y = y.map({"No":0,"Yes":1})

X_train, X_test, y_train, y_test = tts(X, y, 
                                       train_size=.75, 
                                       random_state=321
                                      )

X_test.head(5)

,Age,Gender,Smoking,Hx Smoking,Hx Radiothreapy,Thyroid Function,Physical Examination,Adenopathy,Pathology,Focality,Risk,T,N,M,Stage,Response
134,51,F,No,Yes,No,Euthyroid,Multinodular goiter,No,Papillary,Uni-Focal,Low,T2,N0,M0,I,Excellent
298,42,M,No,No,No,Euthyroid,Single nodular goiter-right,No,Papillary,Multi-Focal,Low,T3a,N0,M0,I,Structural Incomplete
127,56,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Papillary,Uni-Focal,Low,T2,N0,M0,I,Biochemical Incomplete
315,29,F,No,No,No,Euthyroid,Multinodular goiter,Right,Papillary,Multi-Focal,Intermediate,T3a,N1b,M0,I,Structural Incomplete
370,78,M,Yes,Yes,Yes,Clinical Hyperthyroidism,Multinodular goiter,No,Follicular,Multi-Focal,High,T4a,N0,M1,IVB,Structural Incomplete


In [3]:
X_test.Stage.unique()

array(['I', 'IVB', 'II', 'III'], dtype=object)

In [4]:
y_train.value_counts()

Recurred
0    210
1     77
Name: count, dtype: int64

# Define the Data Preprocessing Encoders

In [5]:
# Define preprocessor
num_features = list(X.columns[:1])
cat_features = list(X.columns[1:])

# Define the numerical transformer
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), 
    ("scaler", MinMaxScaler())
])

# Define the categorical transformer
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown="ignore")),
])

# Define your ColumnTransformer (preprocessor)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ],
    remainder="passthrough",
)

pca = PCA(n_components=5)

model = LR(C=0.36, penalty='l1', solver='liblinear')

pipeline = Pipeline(
    steps = [
        ("preprocessor", preprocessor),
        ("pca", pca),
        ("clf", model),
    ])
# Compute sample weights based on class imbalance
sample_weights = compute_sample_weight(class_weight="balanced", y=y_train)

In [6]:
# Define the models using the best parameters from hyperparameter tuning

from functools import partial
metrics = {
    "Balanced Accuracy": balanced_accuracy_score,
    "F1 Score": partial(f1_score, average='weighted'),
    "ROC AUC": roc_auc_score,
    "Sensitivity": partial(recall_score, pos_label=1),
    "Specificity": partial(recall_score, pos_label=0),
    "Precision": partial(precision_score, average='weighted'),
}

# Define the list of metrics for models evaluation using 10-fold CV
ten_fold_CV_metrics = {
    "Balanced Accuracy": "balanced_accuracy",
    "F1 Score": "f1_macro",
    "ROC AUC": "roc_auc",
    "Sensitivity": make_scorer(recall_score, pos_label=1),
    "Specificity":  make_scorer(recall_score, pos_label=0),
    "Precision": "precision"
}

In [7]:
X_test_clinical = X_test.copy()

# Create subgroups
X_test_clinical['AgeGroup'] = pd.cut(X_test_clinical['Age'], bins=[0, 44, 60, 120], labels=['<45', '45–60', '>60'])
X_test_clinical['RiskGroup'] = X_test_clinical['Risk']
X_test_clinical['T_stage'] = X_test_clinical['T']
X_test_clinical['N_stage'] = X_test_clinical['N']
X_test_clinical['M_stage'] = X_test_clinical['M']
X_test_clinical['AdenopathyGroup'] = X_test_clinical['Adenopathy']
X_test_clinical['PathologyGroup'] = X_test_clinical['Pathology']
X_test_clinical['FocalityGroup'] = X_test_clinical['Focality']
X_test_clinical['Hx_RadiothreapyGroup'] = X_test_clinical['Hx Radiothreapy']

In [8]:
def evaluate_pipeline_by_group(pipeline, X, y, group_series, metric_funcs):
    results = {}

    for group in group_series.unique():
        mask = group_series == group
        y_true = y[mask]
        X_group = X.loc[mask]

        if len(y_true.unique()) < 2:
            # Skip this group due to lack of class diversity
            results[group] = {name: "N/A (1 class only)" for name in metric_funcs}
            continue

        y_pred = pipeline.predict(X_group)

        # Some metrics like ROC AUC require probabilities
        if "ROC AUC" in metric_funcs:
            try:
                y_score = pipeline.predict_proba(X_group)[:, 1]
            except:
                y_score = y_pred

        group_metrics = {}
        for name, func in metric_funcs.items():
            try:
                if name == "ROC AUC":
                    score = func(y_true, y_score)
                else:
                    score = func(y_true, y_pred)

                # Round numeric scores to 3 decimal places
                group_metrics[name] = round(score, 3)

            except Exception as e:
                group_metrics[name] = f"Error: {e}"

        results[group] = group_metrics

    return pd.DataFrame(results).T

In [9]:
pipeline.fit(X_train, y_train, clf__sample_weight=sample_weights)

# Evaluate by Age Group
age_group_results = evaluate_pipeline_by_group(pipeline, X_test, y_test, X_test_clinical["AgeGroup"], metrics)

# Evaluate by Risk Group
risk_group_results = evaluate_pipeline_by_group(pipeline, X_test, y_test, X_test_clinical["RiskGroup"], metrics)

# Evaluate by TNM Stage
t_stage_results = evaluate_pipeline_by_group(pipeline, X_test, y_test, X_test_clinical["T_stage"], metrics)
n_stage_results = evaluate_pipeline_by_group(pipeline, X_test, y_test, X_test_clinical["N_stage"], metrics)
m_stage_results = evaluate_pipeline_by_group(pipeline, X_test, y_test, X_test_clinical["M_stage"], metrics)
adenopathy_group_results = evaluate_pipeline_by_group(pipeline, X_test, y_test, X_test_clinical['AdenopathyGroup'], metrics)
pathology_group_results = evaluate_pipeline_by_group(pipeline, X_test, y_test, X_test_clinical['PathologyGroup'], metrics)
focality_group_results = evaluate_pipeline_by_group(pipeline, X_test, y_test, X_test_clinical['FocalityGroup'], metrics)
hx_radiothreapy_group_results = evaluate_pipeline_by_group(pipeline, X_test, y_test, X_test_clinical['Hx_RadiothreapyGroup'], metrics)

# Save the results
age_group_results.to_csv("../results/agegroup_pca_lr_eval.csv")
risk_group_results.to_csv("../results/riskgroup_pca_lr_eval.csv")
t_stage_results.to_csv("../results/tstage_pca_lr_eval.csv")
m_stage_results.to_csv("../results/mstage_pca_lr_eval.csv")
n_stage_results.to_csv("../results/nstage_pca_lr_eval.csv")
adenopathy_group_results.to_csv("../results/adenopathygroup_pca_lr_eval.csv")
pathology_group_results.to_csv("../results/pathologygroup_pca_lr_eval.csv")
focality_group_results.to_csv("../results/focalitygroup_pca_lr_eval.csv")
hx_radiothreapy_group_results.to_csv("../results/hx_radiothreapygroup_pca_lr_eval.csv")

# View results
print("Age Group:")
display(age_group_results)

print("\nRisk Group:")
display(risk_group_results)

print("\nTMN Stage:")
display(t_stage_results)
display(m_stage_results)
display(n_stage_results)

print("\nAdenopathy Group:")
display(adenopathy_group_results)

print("\nPathology Group:")
display(pathology_group_results)

print("\nFocality Group:")
display(focality_group_results)

print("\nHx Radiotherapy:")
display(hx_radiothreapy_group_results)

C:\Users\HP\anaconda3\envs\dl_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\envs\dl_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\envs\dl_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Age Group:


,Balanced Accuracy,F1 Score,ROC AUC,Sensitivity,Specificity,Precision
45–60,0.955,0.942,0.985,1.000,0.909,0.950
<45,0.931,0.927,0.991,0.941,0.920,0.933
>60,0.875,0.913,1.000,1.000,0.750,0.926



Risk Group:


,Balanced Accuracy,F1 Score,ROC AUC,Sensitivity,Specificity,Precision
Low,0.983,0.975,0.983,1.0,0.967,0.989
Intermediate,0.6,0.815,0.957,1.0,0.2,0.878
High,N/A (1 class only),N/A (1 class only),N/A (1 class only),N/A (1 class only),N/A (1 class only),N/A (1 class only)


...

,Balanced Accuracy,F1 Score,ROC AUC,Sensitivity,Specificity,Precision
Papillary,0.938,0.932,0.993,0.962,0.915,0.936
Follicular,1.0,1.0,1.0,1.0,1.0,1.0
Micropapillary,N/A (1 class only),N/A (1 class only),N/A (1 class only),N/A (1 class only),N/A (1 class only),N/A (1 class only)
Hurthel cell,0.667,0.5,1.0,1.0,0.333,0.833



Focality Group:


,Balanced Accuracy,F1 Score,ROC AUC,Sensitivity,Specificity,Precision
Uni-Focal,0.928,0.966,0.995,0.875,0.980,0.966
Multi-Focal,0.821,0.857,0.984,1.000,0.643,0.889



Hx Radiotherapy:


,Balanced Accuracy,F1 Score,ROC AUC,Sensitivity,Specificity,Precision
No,0.937,0.927,0.991,0.966,0.908,0.934
Yes,N/A (1 class only),N/A (1 class only),N/A (1 class only),N/A (1 class only),N/A (1 class only),N/A (1 class only)
